In [2]:
from google.colab import drive  # 구글 코랩에서 drive(내 구글 드라이브)를 사용하기 위한 함수
import io
drive.mount('/content/drive')  # 이렇게 하면 내 drive가 설정? 잡힌다.

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install surprise

     |████████████████████████████████| 11.8MB 7.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670930 sha256=be7456bc41ca62120b3603770a01cdce5242d3bbbd2a2e32fd01d8ed2ce43b48
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
import pandas as pd # pandas 불러오기
import numpy as np # numpy 불러오기
import math # 불러오기 수학과 관련된 함수들
import re #정규표현식 함수 .맞는지 틀린지
from scipy.sparse import csr_matrix   # 매트릭스 해주는거.
import matplotlib.pyplot as plt # 시각화 함수
import seaborn as sns # 시각화 함수
from surprise import Reader, Dataset, SVD, NormalPredictor, KNNBasic
from surprise import KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import BaselineOnly, SVDpp,NMF, SlopeOne, CoClustering #분석툴
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import cross_validate, KFold, train_test_split
import json
import mpmath
from scipy.special import logsumexp

from google.colab import drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
ratings = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/ratings_small.csv',low_memory=False)
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')
ratings.userId = pd.to_numeric(ratings.userId, errors='coerce')
ratings.rating = pd.to_numeric(ratings.rating, errors='coerce')
len(ratings)
df = ratings
print(df)

        userId  movieId  rating
0            1       31     2.5
1            1     1029     3.0
2            1     1061     3.0
3            1     1129     2.0
4            1     1172     4.0
...        ...      ...     ...
99999      671     6268     2.5
100000     671     6269     4.0
100001     671     6365     4.0
100002     671     6385     2.5
100003     671     6565     3.5

[100004 rows x 3 columns]


In [6]:
p = df.groupby('rating')['rating'].agg(['count'])
movie_count = df['movieId'].nunique()
cust_count = df['userId'].nunique()
rating_count = df['userId'].count()
df = df[pd.notnull(df['rating'])]

In [7]:
f = ['count','mean']
df_movie_summary = df.groupby('movieId')['rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int) # map 함수 쓰면 한번에 형변환 처리 가능 , 스트나 튜플을 지정함수로 처리해주는 역할
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)  #quantile 사분위수
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index
print('Movie minimum times of review: {}'.format(movie_benchmark))

Movie minimum times of review: 7.0


In [8]:
df_cust_summary = df.groupby('userId')['rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index
print('Customer minimum times of review: {}'.format(cust_benchmark))

Customer minimum times of review: 138.0


In [13]:
# print('Original Shape: {}'.format(df.shape))
df = df[~df['movieId'].isin(drop_movie_list)] # df의 'Movie_Id'에서 drop_movie_list의 값이 있으면 True
# print('df not isin dropmovie', df)
df = df[~df['userId'].isin(drop_cust_list)]
# print('df not isin dropcust', df)
# print('After Trim Shape: {}'.format(df.shape))
# print('-Data Examples-')
# print(df.iloc[::5000000, :])

In [14]:
df_p = pd.pivot_table(df,values='rating',index='userId',columns='movieId')
# print(df_p)

In [15]:
meta = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)


In [16]:
meta = meta[['id', 'original_title', 'genres','release_date','popularity','original_language']]
meta = meta.rename(columns={'id':'movieId'})
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
meta.popularity = pd.to_numeric(meta.popularity, errors='coerce')   # popularity를 문자열에서 숫자형으로 변환!

# meta.head()

# 여기 까지가 기존 소스 아래부터는 변수 추가 부분과 알고리즘

In [17]:
def user_release_ratio(df, usernumber):
    user_df =df[df['userId'] == usernumber]
    meta2 = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)
    value_meta = meta2[['id','original_title','release_date', 'genres']]

    value_meta = value_meta.rename(columns={'id':'movieId'})
    value_meta.movieId = pd.to_numeric(value_meta.movieId, errors='coerce')
    value_meta = value_meta.dropna(axis=0)
    value_meta = value_meta.reset_index()
    merge_data = pd.merge(user_df, value_meta, on='movieId', how='left')
    merge_data = merge_data.dropna(axis=0)
    merge_data = merge_data.reset_index()

    release_date_list = {'1900':0,'1950':0,'1960':0,'1970':0,'1980':0,'1990':0,'2000':0,'2010':0,'2020':0}
    for i in range(0,len(merge_data)):
        if int(merge_data['release_date'].loc[i][0:4]) <= 1900:
            release_date_list["1900"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1950:
            release_date_list["1950"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1960:
            release_date_list["1960"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1970:
            release_date_list["1970"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1980:
            release_date_list["1980"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1990:
            release_date_list["1990"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 2000:
            release_date_list["2000"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 2010:
            release_date_list["2010"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 2020:
            release_date_list["2020"] += 1
    release_date_list

    sum = 0
    for i in release_date_list:

        sum += release_date_list[i]

    release_date_rate = []
    for i in release_date_list:
        if release_date_list[i] ==0:
            continue
        release_date_list[i] = round((release_date_list[i]/sum),3)
    return release_date_list

In [18]:
def Estimate_Score_sum1(user_df, user_release_ratio_list):
    user_df = user_df.dropna(axis=0)
    for i in range(0,len(user_df)):
        if int(user_df.iloc[i]['release_date'][0:4]) <= 1900:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1900']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1950:
            user_df['Estimate_Score'].loc[user_df.index[i]]= user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1950']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1960:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1960']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1970:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1970']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1980:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1980']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1990:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1990']
        elif int(user_df.iloc[i]['release_date'][0:4])  <= 2000:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['2000']
        elif int(user_df.iloc[i]['release_date'][0:4])  <= 2010:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['2010']
        elif int(user_df.iloc[i]['release_date'][0:4])  <= 2020:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['2020']
    return user_df


In [19]:
def user_pop_ratio(df, usernumber):
    user_df =df[df['userId'] == usernumber]
    meta2 = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)
    value_meta = meta2[['id','original_title', 'popularity','genres']]

    value_meta = value_meta.rename(columns={'id':'movieId'})
    value_meta.movieId = pd.to_numeric(value_meta.movieId, errors='coerce')
    value_meta.popularity = pd.to_numeric(value_meta.popularity, errors='coerce') 
    value_meta = value_meta.dropna(axis=0)
    value_meta = value_meta.reset_index()
    merge_data = pd.merge(user_df, value_meta, on='movieId', how='left')
    merge_data = merge_data.dropna(axis=0)
    merge_data = merge_data.reset_index()

    popularity_list = {'2':0,'6':0,'9':0,'13':0,'18':0,'31':0,'64':0,'185':0,'288':0, '547':0}
    for i in range(0,len(merge_data)):
        if int(merge_data['popularity'].loc[i]) <= 2:
            popularity_list["2"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 6:
            popularity_list["6"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 9:
            popularity_list["9"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 13:
            popularity_list["13"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 18:
            popularity_list["18"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 31:
            popularity_list["31"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 64:
            popularity_list["64"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 185:
            popularity_list["185"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 288:
            popularity_list["288"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 547:
            popularity_list["547"] += 1
    # popularity_list

    sum = 0
    for i in popularity_list:

        sum += popularity_list[i]

    #popularity_list = []
    for i in popularity_list:
        if popularity_list[i] ==0:
            continue
        popularity_list[i] = round((popularity_list[i]/sum),3)
    return popularity_list

In [20]:
def Estimate_Score_sum2(user_df, user_pop_ratio_list):
    user_df = user_df.dropna(axis=0)
    for i in range(0,len(user_df)):
        if int(user_df.iloc[i]['popularity']) <= 2:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['2']
        elif int(user_df.iloc[i]['popularity']) <= 6:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['6']
        elif int(user_df.iloc[i]['popularity']) <= 9:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['9']
        elif int(user_df.iloc[i]['popularity']) <= 13:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['13']
        elif int(user_df.iloc[i]['popularity']) <= 18:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['18']
        elif int(user_df.iloc[i]['popularity']) <= 31:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['31']
        elif int(user_df.iloc[i]['popularity'])  <= 64:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['64']
        elif int(user_df.iloc[i]['popularity'])  <= 185:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['185']
        elif int(user_df.iloc[i]['popularity'])  <= 288:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['288']
        elif int(user_df.iloc[i]['popularity'])  <= 547:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['547']
    return user_df


In [21]:
def user_language_ratio(df, usernumber):
    user_df =df[df['userId'] == usernumber]
    meta2 = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)
    value_meta = meta2[['id','original_title', 'original_language','genres']]

    value_meta = value_meta.rename(columns={'id':'movieId'})
    value_meta.movieId = pd.to_numeric(value_meta.movieId, errors='coerce')
    # value_meta.popularity = pd.to_numeric(value_meta.popularity, errors='coerce') 
    value_meta = value_meta.dropna(axis=0)
    value_meta = value_meta.reset_index()
    merge_data = pd.merge(user_df, value_meta, on='movieId', how='left')
    merge_data = merge_data.dropna(axis=0)
    merge_data = merge_data.reset_index()

    original_language_list = {'en':0,'fr':0,'it':0,'ja':0,'de':0}
    for i in range(0,len(merge_data)):
        if merge_data['original_language'].loc[i] == 'en':
            original_language_list["en"] += 1
        elif merge_data['original_language'].loc[i] == 'fr':
            original_language_list["fr"] += 1
        elif merge_data['original_language'].loc[i] == 'it':
            original_language_list["it"] += 1
        elif merge_data['original_language'].loc[i] == 'ja':
            original_language_list["ja"] += 1
        elif merge_data['original_language'].loc[i] == 'de':
            original_language_list["de"] += 1
       
    # original_language_list

    sum = 0
    for i in original_language_list:

        sum += original_language_list[i]

    # popularity_list = []
    for i in original_language_list:
        if original_language_list[i] ==0:
            continue
        original_language_list[i] = round((original_language_list[i]/sum),3)

    return original_language_list

In [22]:
def Estimate_Score_sum3(user_df, user_language_ratio_list):
    user_df = user_df.dropna(axis=0)
    for i in range(0,len(user_df)):
        if user_df.iloc[i]['original_language'] == 'en':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['en']
        elif user_df.iloc[i]['original_language'] == 'fr':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['fr']
        elif user_df.iloc[i]['original_language'] == 'it':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['it']
        elif user_df.iloc[i]['original_language'] == 'ja':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['ja']
        elif user_df.iloc[i]['original_language'] == 'de':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['de']
       
    return user_df

In [23]:
#우선 사용할 것들을 다시 불러오는 부분
def user_vote_ratio():
  meta = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)
  meta = meta[['id', 'original_title', 'genres','poster_path','vote_average','vote_count']]
  meta = meta.rename(columns={'id':'movieId'})
  meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
  meta.head()
  meta.vote_average =  pd.to_numeric(meta.vote_average, errors='coerce') #데이터프레임의 숫자들이 문자여서 숫자로 바꿨습니다.
  meta.vote_count =  pd.to_numeric(meta.vote_count, errors='coerce')
  meta = meta[['movieId','original_title','poster_path','vote_average','vote_count','genres']].dropna() #결측값 제거
  mata = meta.reset_index(inplace=True,drop=True)
  meta.index = meta.index.map(int)
  
  meta['vote_round'] = ''  # 예를들어 구간을 6.0 6.5 7.0 이런식으로 나눌 예정이므로 추가
  meta['Adj'] = '' #예측치에 가중치를 더하기 위해서 2자리면서 20이면 2.2를 주고 30이면 2.3을 주는 식으로 구성하였습니다. 
  meta.movieId = meta.movieId.astype(int) #이것도 데이터 프레임의 형식을 바꿔주었습니다.
  meta.vote_average = meta.vote_average.astype(float)
  meta.vote_count = meta.vote_count.astype(int)
  print(meta.head())
  meta.info() # 정보를 확인하고 결측값이랑 형식을 다시 한 번 확인합니다. 
  meta.describe() #Adj를 만들기 위해 vote_count의 최댓값을 파악합니다. 자리수가 몇개인지 알기 위해 출력합니다. 5자리가 최대군요.
  
  for idx in range(len(meta)):
      if str(meta.vote_average[idx])[-1] in ["1","2","3","4","6","7","8","9"]:
        meta.vote_round[idx] = round(meta.vote_average[idx],0)
      else:
        meta.vote_round[idx] = meta.vote_average[idx]
  
  
  for idx in range(len(meta)):
      if len(str(meta.vote_count[idx])) == 1:
          meta.Adj[idx] = 1.1 / 250
      if len(str(meta.vote_count[idx])) == 2:
        if str(meta.vote_count[idx])[0] == "1":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.1) / 250
        if str(meta.vote_count[idx])[0] == "2":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.2) / 250
        if str(meta.vote_count[idx])[0] == "3":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.3) / 250
        if str(meta.vote_count[idx])[0] == "4":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.4) / 250
        if str(meta.vote_count[idx])[0] == "5":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.5) / 250
        if str(meta.vote_count[idx])[0] == "6":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.6) / 250
        if str(meta.vote_count[idx])[0] == "7":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.7) / 250
        if str(meta.vote_count[idx])[0] == "8":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.8) / 250
        if str(meta.vote_count[idx])[0] == "9":
          meta.Adj[idx] = (meta.vote_average[idx] * 2.9) / 250
      if len(str(meta.vote_count[idx])) == 3:
        if str(meta.vote_count[idx])[0] == "1":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.1) / 250
        if str(meta.vote_count[idx])[0] == "2":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.2) / 250
        if str(meta.vote_count[idx])[0] == "3":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.3) / 250
        if str(meta.vote_count[idx])[0] == "4":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.4) / 250
        if str(meta.vote_count[idx])[0] == "5":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.5) / 250
        if str(meta.vote_count[idx])[0] == "6":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.6) / 250
        if str(meta.vote_count[idx])[0] == "7":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.7) / 250
        if str(meta.vote_count[idx])[0] == "8":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.8) / 250
        if str(meta.vote_count[idx])[0] == "9":
          meta.Adj[idx] = (meta.vote_average[idx] * 3.9) / 250
      if len(str(meta.vote_count[idx])) == 4:
        if str(meta.vote_count[idx])[0] == "1":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.1) / 250
        if str(meta.vote_count[idx])[0] == "2":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.2) / 250
        if str(meta.vote_count[idx])[0] == "3":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.3) / 250
        if str(meta.vote_count[idx])[0] == "4":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.4) / 250
        if str(meta.vote_count[idx])[0] == "5":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.5) / 250
        if str(meta.vote_count[idx])[0] == "6":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.6) / 250
        if str(meta.vote_count[idx])[0] == "7":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.7) / 250
        if str(meta.vote_count[idx])[0] == "8":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.8) / 250
        if str(meta.vote_count[idx])[0] == "9":
          meta.Adj[idx] = (meta.vote_average[idx] * 4.9) / 250
      if len(str(meta.vote_count[idx])) == 5:
        if str(meta.vote_count[idx])[0] == "1":
          meta.Adj[idx] = (meta.vote_average[idx] * 5.1) / 250
  
  meta['genres'] = meta['genres'].apply(parse_genres)

  data = pd.merge(ratings, meta, on='movieId', how='inner')
  data.info() # 데이터 형식과 결측값 확인하기. 
  data.userId = data.userId.astype('int32') # 데이터 형식 중에서 숫자가 32크기 이하여야 계산이 되더라구요. 그래서 32로 바꿔주기
  data.movieId = data.movieId.astype('int32')
  data.rating	 = data.rating.astype('float32')
  data.vote_average = data.vote_average.astype('float32')
  data.vote_count = data.vote_count.astype('int32')
  data.vote_round = data.vote_round.astype(str).astype('float32')
  data.Adj = pd.to_numeric(data.Adj, errors='coerce').astype('float32')
  return data, meta

In [24]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

In [25]:
# 구한 예측값에 Adj를 추가하기 위해서 만든 함수입니다. 
def Estimate_Score_sum4(df_user,user_df):
  dfsort = df_user.groupby('vote_round')['vote_round'].agg(['count'])
  dfsort = dfsort.sort_values(by='count',ascending = False)
  dfsort["countAdj"] = ''
  dfsort['Rank'] = ''
  for idx in range(len(dfsort)):
    dfsort["countAdj"].iloc[idx] = dfsort['count'].iloc[idx] / 100 
    dfsort['Rank'].iloc[idx] = idx+1 
  dfsort.reset_index(inplace=True)
  for idx in range(len(user_df)):
    for idx2 in range(len(dfsort)):
      if user_df.vote_round.iloc[idx] ==  dfsort.index[idx2]:
          user_df.Adj[idx]+= dfsort.countAdj[idx2]
  for idx in range(len(user_df)):
    user_df.Estimate_Score[idx] += user_df.Adj[idx]

In [26]:
def ALS(df_p):
      R = np.array(df_p) # ALS하기 위해 배열이 필요해서 배열로 바꿔준다.
      R = np.nan_to_num(R) #nan_to_num : nan 값을 0으로 바꾸기 위한 함수
      r_lambda = 40 # 블로그에서 언급하길 논문에서 가장 좋은 결과를 냈다는 값을 가져다 쓴다.
      nf = 200
      alpha = 40

      R = np.array(df_p)# ALS하기 위해 배열이 필요해서 배열로 바꿔준다.
      R = np.nan_to_num(R) #nan_to_num : nan 값을 0으로 바꾸기 위한 함수

      nu = R.shape[0] # nu는 userId의 개수
      ni = R.shape[1] # ni는 movieId의 개수 

      # initialize X and Y with very small values
      X = np.random.rand(nu, nf) * 0.01   # np.random.rand : 0 ~ 1의 균일분포 표준정규분포 난수를 matrix array(m(행),n(컬럼)) 생성
      Y = np.random.rand(ni, nf) * 0.01

      P = np.copy(R) 
      P[P > 0] = 1  #위의 사용자 평점들을 1로 바꿔줘야 하기 때문에 0보다 크면 1로 다 바꿔준다.
      C = 1 + alpha * R

      predict_errors = []
      confidence_errors = []
      regularization_list = []
      total_losses = []

      for i in range(2):
        if i != 0:
            yT = np.transpose(Y) # np.transpose 행과 열을 바꾸는 함수
            for u in range(nu):
                Cu = np.diag(C[u]) # np.diag 대각행렬 만드는 함수
                yT_Cu_y = np.matmul(np.matmul(yT, Cu), Y) # np.matmul 두 배열의 행렬곱
                lIy = np.dot(r_lambda, np.identity(nf)) # np.dot 두 배열의 내적곱, np.identity 2차원의 정방단위행렬 객체를 반환 
                yT_Cu_pu = np.matmul(np.matmul(yT, Cu), P[u])
                X[u] = np.linalg.solve(yT_Cu_y + lIy, yT_Cu_pu) # np.linalg.solve 연립방정식 해 풀기.

            xT = np.transpose(X)
            for i in range(ni):
                Ci = np.diag(C[:, i])
                xT_Ci_x = np.matmul(np.matmul(xT, Ci), X)
                lIx = np.dot(r_lambda, np.identity(nf))
                xT_Ci_pi = np.matmul(np.matmul(xT, Ci), P[:, i])
                Y[i] = np.linalg.solve(xT_Ci_x + lIx, xT_Ci_pi)
        predict = np.matmul(X, np.transpose(Y))
        predict_error = np.square(P - predict)
        confidence_error = np.sum(C * predict_error)
        regularization = r_lambda * (np.sum(np.square(X)) + np.sum(np.square(Y)))
        total_loss = confidence_error + regularization
        # predict_error, confidence_error, regularization, total_loss = loss_function(C, P, predict, X, Y, r_lambda)

        predict_errors.append(predict_error)
        confidence_errors.append(confidence_error)
        regularization_list.append(regularization)
        total_losses.append(total_loss)

      predict = np.matmul(X, np.transpose(Y))
      print('final predict')
      print([predict])

      return  predict

In [27]:
reader = Reader()

알고리즘 별 cross_validate


In [29]:
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
svd = SVD()
slope = SlopeOne()
nmf = NMF()

bsl_options = {'method': 'als',
'n_epochs': 5,
'reg_u': 12,
'reg_i': 5
}
als = BaselineOnly(bsl_options=bsl_options)

als_result = cross_validate(als, data, measures=['RMSE', 'MAE'],cv=5, verbose=False) #evaluate 대신 cross_validate
slope_result = cross_validate(slope, data, measures=['RMSE', 'MAE'],cv=5, verbose=False) #evaluate 대신 cross_validate
svd_result = cross_validate(svd, data, cv = 5,  measures=['RMSE', 'MAE']) #evaluate 대신 cross_validate사용
nmf_result = cross_validate(nmf, data, measures=['RMSE', 'MAE'],cv=5, verbose=False) #evaluate 대신 cross_validate

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


#알고리즘에 따른 RMSE, MAE

In [30]:
print(slope_result)
print(svd_result)
print(nmf_result)
print(als_result)

{'test_rmse': array([0.86872043, 0.88205856, 0.87578982, 0.87326189, 0.87864439]), 'test_mae': array([0.66716229, 0.67620641, 0.67407651, 0.67132772, 0.67535656]), 'fit_time': (1.252148151397705, 1.3201446533203125, 1.2508783340454102, 1.2572457790374756, 1.2340633869171143), 'test_time': (4.2021002769470215, 4.108436107635498, 4.224747896194458, 4.2137181758880615, 4.12137246131897)}
{'test_rmse': array([0.86916962, 0.86291156, 0.86599705, 0.87134579, 0.86378004]), 'test_mae': array([0.66750235, 0.66131805, 0.66982297, 0.67046253, 0.66461868]), 'fit_time': (3.1548147201538086, 3.183074474334717, 3.073922634124756, 3.1499502658843994, 3.1011548042297363), 'test_time': (0.08562612533569336, 0.16008591651916504, 0.08545637130737305, 0.08406424522399902, 0.16350865364074707)}
{'test_rmse': array([0.89304698, 0.89542991, 0.89546245, 0.89101766, 0.90897443]), 'test_mae': array([0.68978134, 0.69239015, 0.6881386 , 0.68613962, 0.70010724]), 'fit_time': (3.223079204559326, 3.1919336318969727, 

# 유저에 따른 개인 영화 추천

In [31]:
def user_difference(data,usernumber,rating,moviedata,dropdata,reader,svd):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    print(df_user)

    user_release_ratio_list = user_release_ratio(df, usernumber) # 유저의 년도 비율을 가져온다.

    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    svd.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: svd.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    print(user_df.head(10))

    return user_df
user_df665 = user_difference(df,665,5,meta,drop_movie_list,reader,svd)
user_df664 = user_difference(df,664,5,meta,drop_movie_list,reader,svd)


movieId
168                                       Jeffrey
317                             Fresa y chocolate
318                          Swimming with Sharks
364            Mrs. Parker and the Vicious Circle
527                                    Serial Mom
587               Snow White and the Seven Dwarfs
595                         A Great Day in Harlem
596                                  Bye Bye Love
597                                  One Fine Day
912                       The Reluctant Debutante
1073       Children of the Corn IV: The Gathering
1097                            Manon des Sources
1101                                Hustler White
1136                   La double vie de Véronique
1270                            I'm Not Rappaport
1704                                  Wild Things
1754                                   Species II
2081                    Why Do Fools Fall In Love
2087                                    Knock Off
2125                                  Simo

In [32]:
print(user_df665)
print(user_df664)

               original_title  ... Estimate_Score
33913              Der Tunnel  ...       4.365413
286        Once Were Warriors  ...       4.297585
534      Sleepless in Seattle  ...       4.285010
8316   Die Büchse der Pandora  ...       4.269557
3244        Dog Day Afternoon  ...       4.264056
...                       ...  ...            ...
11993                 Abraham  ...       2.165204
19332                   薔薇の葬列  ...       2.064001
11324             Dr. Cyclops  ...       2.007103
1207                Chinatown  ...       1.975700
7718   The Day After Tomorrow  ...       1.902223

[43759 rows x 6 columns]
                           original_title  ... Estimate_Score
11566                       Lonely Hearts  ...       4.573478
6388   Terminator 3: Rise of the Machines  ...       4.571144
11922                      License to Wed  ...       4.561388
2649              The Thomas Crown Affair  ...       4.528443
6910                  Birdman of Alcatraz  ...       4.514169
..

# 유저의 변화에 따른 영화추천

In [74]:
def user_difference(data,usernumber,rating,moviedata,dropdata,reader,svd):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    print(df_user)

    user_release_ratio_list = user_release_ratio(df, usernumber) # 유저의 년도 비율을 가져온다.

    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    svd.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: svd.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    print(user_df.head(10))

    return user_df
user_1 = user_difference(df,1,5,meta,drop_movie_list,reader,svd)
df2 = df
df2.loc[1] = [1,5502,5.0]
df2.loc[2] = [1,5991.0,5.0]
add_user_1 = user_difference(df2,1,5,meta,drop_movie_list,reader,svd)


Series([], Name: original_title, dtype: object)
                           original_title  ... Estimate_Score
534                  Sleepless in Seattle  ...       4.407289
11566                       Lonely Hearts  ...       4.375443
4020             The Million Dollar Hotel  ...       4.348210
6836                      Broken Blossoms  ...       4.313123
6388   Terminator 3: Rise of the Machines  ...       4.305612
2648              The Thomas Crown Affair  ...       4.302566
2649              The Thomas Crown Affair  ...       4.295724
3058                         Galaxy Quest  ...       4.291136
22347                               Miffo  ...       4.289086
6141                       The Good Thief  ...       4.287707

[10 rows x 6 columns]
movieId
5502.0    Gojira-Minira-Gabara: Oru kaijû daishingeki
5991.0                                      Six Weeks
Name: original_title, dtype: object
                 original_title  ... Estimate_Score
6141             The Good Thief  ...       

# 변수 별 영화 추천

In [33]:
def variable_weight(data,usernumber,rating,moviedata,dropdata,reader,algo):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    # print(df_user)

    user_release_ratio_list = user_release_ratio(df, usernumber) # 유저의 년도 비율을 가져온다.
    # print(user_release_ratio_list)
    user_pop_ratio_list = user_pop_ratio(df, usernumber) # 유저의 popularity 비율을 가져온다.
    # print(user_pop_ratio_list)
    user_language_ratio_list = user_language_ratio(df, usernumber) # 유저의 language 비율을 가져온다.
    # print(user_language_ratio_list)

    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    algo.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: algo.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    # print(user_df.head(10))

    user_df_sum = Estimate_Score_sum1(user_df, user_release_ratio_list)
    user_df_sum_relase = user_df_sum.sort_values('Estimate_Score', ascending=False)
    print("개봉일 별 가중치")
    print(user_df_sum_relase.head(10))

    user_df_sum = Estimate_Score_sum2(user_df, user_pop_ratio_list)
    user_df_sum_pop = user_df_sum.sort_values('Estimate_Score', ascending=False)
    print("인기 별 가중치")
    print(user_df_sum_pop.head(10))

    user_df_sum = Estimate_Score_sum3(user_df, user_language_ratio_list)
    user_df_sum_lang = user_df_sum.sort_values('Estimate_Score', ascending=False)
    print("언어 별 가중치")
    print(user_df_sum_lang.head(10))


    return user_df_sum_relase, user_df_sum_pop, user_df_sum_lang
user_df_sum_relase,user_df_sum_pop, user_df_sum_lang = variable_weight(df,665,5,meta,drop_movie_list,reader,svd)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_lau

개봉일 별 가중치
                             original_title  ... Estimate_Score
24036                        Der rote Elvis  ...       4.703604
10942                           Silent Hill  ...       4.686523
10089                            Madagascar  ...       4.671458
11566                         Lonely Hearts  ...       4.626606
3058                           Galaxy Quest  ...       4.626372
6399                            Bad Boys II  ...       4.613243
11125                 Stranger Than Fiction  ...       4.611682
11481                    Notes on a Scandal  ...       4.606076
37719  Confession of a Child of the Century  ...       4.602485
22347                                 Miffo  ...       4.588020

[10 rows x 6 columns]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykerne

인기 별 가중치
                             original_title  ... Estimate_Score
37719  Confession of a Child of the Century  ...       4.795485
938                            The 39 Steps  ...       4.751290
9758                  Le Clan des Siciliens  ...       4.594746
6836                        Broken Blossoms  ...       4.593649
700                                Dead Man  ...       4.564389
5295                            5 Card Stud  ...       4.561316
24036                        Der rote Elvis  ...       4.536604
10942                           Silent Hill  ...       4.524523
8737                        Cousin, Cousine  ...       4.521144
33913                            Der Tunnel  ...       4.518110

[10 rows x 6 columns]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykerne

언어 별 가중치
          original_title  ... Estimate_Score
938         The 39 Steps  ...       5.363290
3058        Galaxy Quest  ...       5.227372
5295         5 Card Stud  ...       5.208316
6836     Broken Blossoms  ...       5.205649
700             Dead Man  ...       5.176389
10942        Silent Hill  ...       5.166523
10089         Madagascar  ...       5.151458
6924             Bullitt  ...       5.123991
3244   Dog Day Afternoon  ...       5.120516
10373            Wedlock  ...       5.114618

[10 rows x 6 columns]


In [34]:
vote_data, vote_meta = user_vote_ratio()

   movieId               original_title  ... vote_round  Adj
0      862                    Toy Story  ...                
1     8844                      Jumanji  ...                
2    15602             Grumpier Old Men  ...                
3    31357            Waiting to Exhale  ...                
4    11862  Father of the Bride Part II  ...                

[5 rows x 8 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 45077 entries, 0 to 45076
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movieId         45077 non-null  int64  
 1   original_title  45077 non-null  object 
 2   poster_path     45077 non-null  object 
 3   vote_average    45077 non-null  float64
 4   vote_count      45077 non-null  int64  
 5   genres          45077 non-null  object 
 6   vote_round      45077 non-null  object 
 7   Adj             45077 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to b

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44958 entries, 0 to 44957
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   userId          44958 non-null  int64  
 1   movieId         44958 non-null  int64  
 2   rating          44958 non-null  float64
 3   original_title  44958 non-null  object 
 4   poster_path     44958 non-null  object 
 5   vote_average    44958 non-null  float64
 6   vote_count      44958 non-null  int64  
 7   genres          44958 non-null  object 
 8   vote_round      44958 non-null  object 
 9   Adj             44958 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 3.8+ MB


In [36]:
#이게 맨 처음에 SVD에 가중치 준 함수입니다. 여기서도 svd, reader, dero_movie_list는 그냥 함수안에 바로 넣었습니다.
def variable_weight2(data,usernumber,rating,moviedata):
    df = data.copy()
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    print(df_user.iloc[:,2]) #제가 만든 meta에 이미 title이 있어서join을 할 필요가 없었습니다.
    
    
    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(drop_movie_list)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    svd.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: svd.predict(usernumber, x).est)
    user_df.reset_index(inplace=True, drop=True)
   
    
    Estimate_Score_sum4(df_user,user_df) 
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values(('Estimate_Score'), ascending=False)
    user_df_sum_vote = pd.DataFrame(user_df.head(10))
    user_df_sum_vote = user_df_sum_vote.iloc[:,[0,2,3,4,6,7]]
    


    return user_df_sum_vote
user_df_sum_vote = variable_weight2(vote_data,665,5,vote_meta)

movieId
168              Star Trek IV: The Voyage Home
317                                   Grbavica
364                             Batman Returns
527                         Once Were Warriors
587                                   Big Fish
318                   The Million Dollar Hotel
595                      To Kill a Mockingbird
2959                            License to Wed
596                        The Grapes of Wrath
1073                            Arlington Road
2087                               The Getaway
2141                                James Dean
2640                                  Heathers
3034                        Young Frankenstein
597                                    Titanic
4025                                 The Prize
2139    Cleopatra Jones and the Casino of Gold
3578                                Der Tunnel
4896                          Muxmäuschenstill
4993                               5 Card Stud
2125                                   Wedlock
912  

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [37]:
print(user_df_sum_relase.head(10))
print(user_df_sum_pop.head(10)) 
print(user_df_sum_lang.head(10))
print(user_df_sum_vote.head(10))

                             original_title  ... Estimate_Score
24036                        Der rote Elvis  ...       4.703604
10942                           Silent Hill  ...       4.686523
10089                            Madagascar  ...       4.671458
11566                         Lonely Hearts  ...       4.626606
3058                           Galaxy Quest  ...       4.626372
6399                            Bad Boys II  ...       4.613243
11125                 Stranger Than Fiction  ...       4.611682
11481                    Notes on a Scandal  ...       4.606076
37719  Confession of a Child of the Century  ...       4.602485
22347                                 Miffo  ...       4.588020

[10 rows x 6 columns]
                             original_title  ... Estimate_Score
37719  Confession of a Child of the Century  ...       4.795485
938                            The 39 Steps  ...       4.751290
9758                  Le Clan des Siciliens  ...       4.594746
6836             

# 알고리즘 별 영화 추천

In [44]:
def userRec3(data,usernumber,rating,moviedata,dropdata,reader,algo):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    # print(df_user)

    user_release_ratio_list = user_release_ratio(df, usernumber) # 유저의 년도 비율을 가져온다.
    # print(user_release_ratio_list)
    user_pop_ratio_list = user_pop_ratio(df, usernumber) # 유저의 popularity 비율을 가져온다.
    # print(user_pop_ratio_list)
    user_language_ratio_list = user_language_ratio(df, usernumber) # 유저의 language 비율을 가져온다.
    # print(user_language_ratio_list)

    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    algo.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: algo.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    # print(user_df.head(10))

    # user_df_sum = Estimate_Score_sum1(user_df, user_release_ratio_list)
    # # user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    # # print(user_df_sum.head(10))

    # user_df_sum = Estimate_Score_sum2(user_df_sum, user_pop_ratio_list)
    # # user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    # # print(user_df_sum.head(10))

    # user_df_sum = Estimate_Score_sum3(user_df_sum, user_language_ratio_list)
    # user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    # print(user_df_sum.head(10))

    return user_df
als_recomend = userRec3(df,665,5,meta,drop_movie_list,reader,als)
svd_recomend = userRec3(df,665,5,meta,drop_movie_list,reader,svd)
slope_recomend = userRec3(df,665,5,meta,drop_movie_list,reader,slope)
nmf_recomend = userRec3(df,665,5,meta,drop_movie_list,reader,nmf)

Estimating biases using als...


In [48]:
# print(als_recomend)
# print(svd_recomend)
# print(slope_recomend)
# print(nmf_recomend)

                     original_title  ... Estimate_Score
4551                 Vampire's Kiss  ...       5.000000
7057          My Darling Clementine  ...       5.000000
22094              End of the World  ...       4.942514
4142              The Mummy Returns  ...       4.842446
927    Mr. Smith Goes to Washington  ...       4.833911
...                             ...  ...            ...
11329                  The Grudge 2  ...       1.484810
3306                   Jungle Fever  ...       1.431271
5672                         8 Mile  ...       1.388452
11324                   Dr. Cyclops  ...       1.305226
1207                      Chinatown  ...       1.031001

[43759 rows x 6 columns]
